Nama : Ariel Cristo Haryono

NIM : 0706022110032

In [14]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from mlxtend.frequent_patterns import apriori, association_rules
! pip install mlxtend

# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

you can refer to this article to find out about apriori and association rule:
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here: 
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [15]:
# load the data set and show the first five transaction

url = "https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv"
df = pd.read_csv(url)
print(df.head())

        0       1     2       3       4       5       6
0   Bread    Wine  Eggs    Meat  Cheese  Pencil  Diaper
1   Bread  Cheese  Meat  Diaper    Wine    Milk  Pencil
2  Cheese    Meat  Eggs    Milk    Wine     NaN     NaN
3  Cheese    Meat  Eggs    Milk    Wine     NaN     NaN
4    Meat  Pencil  Wine     NaN     NaN     NaN     NaN


# Get the set of product that has been purchased


Get the unique product that has been purchased

In [16]:
unique_products = pd.unique(df.values.ravel())
print("Unique Products Purchased:")
print(set(unique_products))

Unique Products Purchased:
{nan, 'Pencil', 'Cheese', 'Eggs', 'Meat', 'Milk', 'Bagel', 'Wine', 'Diaper', 'Bread'}


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [17]:
#create an itemset based on the products
itemset_df = pd.get_dummies(df, prefix='', prefix_sep='').groupby(axis=1, level=0).max()

# encoding the feature
encoded_vals = []
for index, row in df.iterrows(): 
    labels = {}
    uncommons = list(set(unique_products) - set(row))
    commons = list(set(unique_products).intersection(row))
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded_vals.append(labels)


In [18]:
  # create new dataframe from the encoded features
encoded_df = pd.DataFrame(encoded_vals)

# show the new dataframe
encoded_df

,NaN,Bagel,Milk,Pencil,Cheese,Eggs,Meat,Wine,Diaper,Bread
0,0,0,0,1,1,1,1,1,1,1
1,0,0,1,1,1,0,1,1,1,1
2,1,0,1,0,1,1,1,1,0,0
3,1,0,1,0,1,1,1,1,0,0
4,1,0,0,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...
310,1,0,0,0,1,1,0,0,0,1
311,1,0,1,1,0,0,1,0,0,0
312,0,0,0,1,1,1,1,1,1,1
313,1,0,0,0,1,0,1,0,0,0


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

In [19]:
encoded_df = encoded_df.dropna(axis=1)


## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products. 
For this case study, we will min_support=0.2

In [20]:
frequently_product = apriori(encoded_df, min_support=0.2, use_colnames=True)
frequently_product

c:\Users\Ariel\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.869841,(nan)
1,0.425397,(Bagel)
2,0.501587,(Milk)
3,0.361905,(Pencil)
4,0.501587,(Cheese)
5,0.438095,(Eggs)
6,0.476190,(Meat)
7,0.438095,(Wine)
8,0.406349,(Diaper)
9,0.504762,(Bread)


Then, we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [21]:
association_rules(frequently_products, metric = "confidence", min_threshold = 0.6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Bagel),(nan),0.425397,0.869841,0.336508,0.791045,0.909413,-0.033520,0.622902,-0.147743
1,(Milk),(nan),0.501587,0.869841,0.409524,0.816456,0.938626,-0.026778,0.709141,-0.115976
2,(Pencil),(nan),0.361905,0.869841,0.266667,0.736842,0.847100,-0.048133,0.494603,-0.220499
3,(Cheese),(nan),0.501587,0.869841,0.393651,0.784810,0.902245,-0.042651,0.604855,-0.178565
4,(Eggs),(nan),0.438095,0.869841,0.336508,0.768116,0.883053,-0.044565,0.561310,-0.190735
5,(Meat),(nan),0.476190,0.869841,0.368254,0.773333,0.889051,-0.045956,0.574230,-0.192405
6,(Wine),(nan),0.438095,0.869841,0.317460,0.724638,0.833069,-0.063613,0.472682,-0.262869
7,(Diaper),(nan),0.406349,0.869841,0.317460,0.781250,0.898152,-0.035999,0.595011,-0.160381
8,(Bread),(nan),0.504762,0.869841,0.396825,0.786164,0.903801,-0.042237,0.608683,-0.176903
9,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,0.064641,1.442650,0.402687


Provide explanation about __antecedent support__, __consequent support__, __support__, __confidence__, __lift__, __leverage__ and __conviction__

1. Antecedent Support:
-The support of the antecedent is the proportion of transactions in the dataset that contain the antecedent (the left-hand side of the rule). It represents the probability of occurrence of the antecedent.

2.Consequent Support:
-Similar to antecedent support, the support of the consequent is the proportion of transactions in the dataset that contain the consequent (the right-hand side of the rule). It represents the probability of occurrence of the consequent.

3.Support:
-Support measures the proportion of transactions in the dataset that contain both the antecedent and the consequent. It represents the joint probability of the antecedent and consequent occurring together.

4.Confidence:
-Cnfidence is a measure of how often the rule has been found to be true. It is the ratio of the support of both the antecedent and consequent to the support of the antecedent alone. It reflects the likelihood that the consequent will be purchased when the antecedent is purchased.

5.Lift:
-Lift measures the strength of the association rule over the expected randomness. It is the ratio of the observed support to the expected support if the antecedent and consequent were independent. A lift value greater than 1 indicates that the rule has a positive effect, while a lift less than 1 suggests a negative effect.

6.Leverage:
-Leverage measures the difference between the observed frequency of antecedent and consequent co-occurrence and the frequency that would be expected if they were independent. It indicates whether the presence of the antecedent and consequent together occurs more often than expected by chance.

7.Conviction:
-Conviction is a measure of how much more likely the consequent is given that the antecedent is true, compared to if they were independent. It takes into account the ratio of the expected false positives to the observed false positives.

Summary:
These metrics provide insights into the strength, significance, and characteristics of association rules mined from transaction data. They help in understanding the reliability and potential impact of the discovered rules on decision-making.